In [1]:
import os

In [2]:
%pwd

'c:\\Users\\harsh\\Desktop\\My_space\\Projects_2025\\DL_Image_Classification_MLFlow_DVC\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\harsh\\Desktop\\My_space\\Projects_2025\\DL_Image_Classification_MLFlow_DVC'

In [5]:
import dagshub
dagshub.init(repo_owner='harshadhumane18', repo_name='DL-Image-Classification-MLFlow-DVC', mlflow=True)

# import mlflow
# with mlflow.start_run():
#   mlflow.log_param('parameter name', 'value')
#   mlflow.log_metric('metric name', 1)

Accessing as harshadhumane18

Initialized MLflow to track repo "harshadhumane18/DL-Image-Classification-MLFlow-DVC"

Repository harshadhumane18/DL-Image-Classification-MLFlow-DVC initialized!

In [6]:
import tensorflow as tf

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [7]:
model = tf.keras.models.load_model("artifacts/training/model.keras")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.keras",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/harshadhumane18/DL-Image-Classification-MLFlow-DVC.mlflow/",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [13]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)
        
        
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )

            # Save & log model manually (DagsHub compatible)
            model_path = "model.keras"
            self.model.save(model_path)
            mlflow.log_artifact(model_path, artifact_path="model")

In [14]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-11-13 23:12:04,830: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-11-13 23:12:04,832: INFO: common: yaml file: params.yaml loaded successfully]
[2025-11-13 23:12:04,834: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 13s 2s/step - loss: 0.2985 - accuracy: 0.8333
[2025-11-13 23:12:20,271: INFO: common: json file saved at: scores.json]
🏃 View run delightful-sow-410 at: https://dagshub.com/harshadhumane18/DL-Image-Classification-MLFlow-DVC.mlflow/#/experiments/0/runs/757d417e9e454901a7d9ddcd4712dc06
🧪 View experiment at: https://dagshub.com/harshadhumane18/DL-Image-Classification-MLFlow-DVC.mlflow/#/experiments/0
